# Inspect

> (Notebooke only) Interactive exploration of trained encoder and embeddings

In [ ]:
import torch
from torch.utils.data import DataLoader
from omegaconf import OmegaConf
from midi_rae.vit import ViTEncoder
from midi_rae.data import PRPairDataset
from midi_rae.viz import make_emb_viz

## Config

In [ ]:
#| eval: false
cfg = OmegaConf.load('../configs/config.yaml')
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
print(f'device = {device}')

device = cpu


## Load Dataset

In [ ]:
#| eval: false
val_ds = PRPairDataset(split='val', max_shift_x=cfg.training.max_shift_x, max_shift_y=cfg.training.max_shift_y)
val_dl = DataLoader(val_ds, batch_size=cfg.training.batch_size, num_workers=4, drop_last=True)
print(f'Loaded {len(val_ds)} validation samples')

## Load Encoder from Checkpoint

In [ ]:
#| eval: false
ckpt_path = '../checkpoints/enc_best.pt'  # <-- change as needed

model = ViTEncoder(cfg.data.in_channels, (cfg.data.image_size, cfg.data.image_size), 
                   cfg.model.patch_size, cfg.model.dim, cfg.model.depth, cfg.model.heads).to(device)

ckpt = torch.load(ckpt_path, map_location=device)
model.load_state_dict(ckpt['model_state_dict'])
model.eval()
print(f'Loaded checkpoint from {ckpt_path}, epoch {ckpt.get("epoch", "?")}')

## Run Batch Through Encoder

In [ ]:
#| eval: false
batch = next(iter(val_dl))
img1, img2 = batch['img1'].to(device), batch['img2'].to(device)

with torch.no_grad():
    z1, pmask1 = model(img1, return_cls_only=False)
    z2, pmask2 = model(img2, return_cls_only=False)

z1 = z1.reshape(-1, z1.shape[-1])
z2 = z2.reshape(-1, z2.shape[-1])
num_tokens = z1.shape[0] // len(batch['img1'])

print(f'z1: {z1.shape}, z2: {z2.shape}, num_tokens: {num_tokens}')

## Visualize Embeddings

In [ ]:
#| eval: false

# Combined embeddings from both views
zs = torch.cat((z1, z2), dim=0)
pmask = torch.cat([pmask1, pmask2], dim=0)
file_idx = batch['file_idx']

# Interactive visualization (without wandb logging)
import plotly.io as pio
pio.renderers.default = 'notebook'
from midi_rae.viz import umap_project, pca_project, plot_embeddings_3d


coords = pca_project(zs.cpu())
fig = plot_embeddings_3d(coords, title='Embeddings (PCA)')
fig.show()

## Inspect Individual Images

In [ ]:
#| eval: false

import matplotlib.pyplot as plt

# Show a sample image pair
idx = 0
fig, axes = plt.subplots(1, 2, figsize=(8, 4))
axes[0].imshow(img1[idx, 0].cpu(), cmap='gray')
axes[0].set_title(f'Image 1 (file_idx={file_idx[idx].item()})')
axes[1].imshow(img2[idx, 0].cpu(), cmap='gray')
axes[1].set_title(f'Image 2')
plt.tight_layout()
plt.show()